In [ ]:
import os
import re
import copy
import pickle
import datetime
from collections import deque

import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import block_reduce
from skimage.transform import resize

import importlib
from IPython.display import clear_output

import data_utils
import CGAN
import Params

## Load and preprocess the data

In [ ]:
with open("./data/scaler.pkl", "rb") as f:
    scaler = pickle.load(f)
with open("./data/table.pkl", "rb") as f:
    lookup_table = pickle.load(f)
face_data = np.load("./data/face_data.npy")
landmarks = np.load("./data/landmarks.npy")
labels = np.load("./data/labels.npy")

In [ ]:
# Visualize the data
data_utils.visualize_z(face_data, z_channel=2)

## Train the model

In [ ]:
import CGAN
import Params

importlib.reload(Params);
importlib.reload(CGAN);
importlib.reload(data_utils);

Params = Params.Params
CGAN = CGAN.CGAN

In [ ]:
X = face_data[:, :, :, 2:3]
X_cond = labels

In [ ]:
def train(M, X, X_cond=None, train_steps=20, prev_steps=0, interval=5, suffix="Recent"):
    
    with open("./models/CGAN-{}-config.txt".format(suffix), "wt") as f:
        f.write(str(M))
    
    history = {
        "loss_D_real": [],
        "loss_D_fake": [],
        "loss_GD": [],
        "acc_D_real": [],
        "acc_D_fake": [],
        "acc_GD": []
    }
    
    for step in range(1, train_steps + 1):

        # Train the discriminator
        for i in range(M.params.steps_D):
            
            indices = np.random.randint(X.shape[0], size=(M.params.batch_size))
            noise = np.random.uniform(size=(M.params.batch_size, M.params.n_rand))
            if X_cond is None:
                z = noise
            else:
                z = np.concatenate([noise, X_cond[indices]], axis=1)
                
            X_real = X[indices]
            X_fake = M.G.predict(z)
            
            Y_real = np.zeros((X_real.shape[0], 1)) + M.params.real_l
            Y_fake = np.zeros((X_fake.shape[0], 1)) + M.params.fake_l

            loss_D_real, acc_D_real = M.D.train_on_batch(X_real, Y_real)
            loss_D_fake, acc_D_fake = M.D.train_on_batch(X_fake, Y_fake)

        # Train the generator
        for i in range(M.params.steps_GD):
            
            noise = np.random.uniform(size=(M.params.batch_size, M.params.n_rand))
            if X_cond is None:
                X_GD = noise
            else:
                label_mat = np.zeros((M.params.batch_size, X_cond.shape[1]))
                label_idx = np.random.randint(X_cond.shape[1], size=(M.params.batch_size))
                label_mat[np.arange(M.params.batch_size), label_idx] = 1
                X_GD = np.concatenate([noise, label_mat], axis=1)

            Y_GD = np.zeros((X_GD.shape[0], 1)) + M.params.real_l
            loss_GD, acc_GD = M.GD.train_on_batch(X_GD, Y_GD)
            
        # Update history for statistics
        history["loss_D_real"].append(loss_D_real)
        history["loss_D_fake"].append(loss_D_fake)
        history["loss_GD"].append(loss_GD)
        history["acc_D_real"].append(acc_D_real)
        history["acc_D_fake"].append(acc_D_fake)
        history["acc_GD"].append(acc_GD)

        # Clear the display output and display window statistics (graph)
        if step % (10 * interval) == 0:
            clear_output(wait=True)
            data_utils.visualize_history(history)

        # Print window statistics
        if step % interval == 0:
            print("Step {}:".format(step + prev_steps))

            data_utils.visualize_z(X_fake)

            print("Descriminator (real) :: loss = {}, acc = {}".format(np.mean(history["loss_D_real"][-10:]),
                                                                       np.mean(history["acc_D_real"][-10:])))
            print("Descriminator (fake) :: loss = {}, acc = {}".format(np.mean(history["loss_D_fake"][-10:]),
                                                                       np.mean(history["acc_D_fake"][-10:])))
            print("Adversarial          :: loss = {}, acc = {}".format(np.mean(history["loss_GD"][-10:]),
                                                                       np.mean(history["acc_GD"][-10:])))
            print()
        
        if (step + prev_steps) % 500 == 0:
            with open("./models/CGAN-{}-{}-model.pkl".format(suffix, step + train_steps), "wb") as f:
                pickle.dump(M, f)
            with open("./models/CGAN-{}-{}-history.pkl".format(suffix, step + train_steps), "wb") as f:
                pickle.dump(history, f)

    return history

In [ ]:
network = CGAN(Params(X))
history = train(network, X, interval=10, train_steps=4000, prev_steps=0, suffix="dcgan_rand")

## Things to try

* Penalize overconfidence by labeling real images with 0.9 (Better GAN training)

* Switch to Adam optimizer (DCGAN paper)

* Switch from max-pool and upsampling to convolution stride (DCGAN)

* Add conditional data to random noise (consider label as one hot) to account for different topology of the data, impede "averaging" of faces w.r.t. facial expressions. (Info-GAN ??)

* Remove dropout from generator (not done)

* Make discriminator more complex (harder to learn, smarter, made same level as generator)

In [ ]:
for fname in sorted(os.listdir("./models/")):
    if fname.endswith(".pkl"):
        with open(os.path.join("./models/", fname), "rb") as f:
            model = pickle.load(f)
            X_fake = model.G.predict(np.random.uniform(size=(model.params.batch_size, model.params.n_rand)))
            print("Model:", fname)
            data_utils.visualize_z(X_fake)
            del model